# Money Mule detection using Neo4j and Scikit-Learn/Tensorflow

## Dataset
Paysim is an approach using an agent-based model and some anonymized, aggregate transactional data from a real mobile money network operator to create synthetic financial data sets academics and hackers can use for exploring ways to detect fraudulent behavior.

More info here: https://www.sisu.io/posts/paysim/

https://www.kaggle.com/kartik2112/fraud-detection-on-paysim-dataset

## Mules

Money mules are defined as customers/clients that collude with fraudsters and facilitate moving money out of the network. Some of the typical activities of money mules are lending their credentials to fraudsters, send money to fraudsters in untreaceable amounts over a period of time, receive money from fraudsters and move money out of network etc.

In this dataset, some clients are labelled as mules. Our task is to train supervised ML classification models using these labelled examples and detect if there are any mules among unlabelled clients in the dataset. 

## What we do
- We will use Neo4j to load the data into a graph. 
- Preprocess the data to generate additional topological features
- Generate graph embeddings using Neo4j GDS library 
- Train a supervised classification model in Neo4j GDS library
- Train a supervised classification model using Keras/Tensorflow

## Why Graphs and Graph Embeddings

Detecting mules using abstract features such as transaction amounts, type of transactions, historical fradulent transactions ets is not effective and leads to a lot of false positives. Mules by definition are clients that collude fraudsters. There are no labelled fraudsters in the dataset. 

Here we need a method to generate a vector representation for every client based on their relationship to other clients and specifically with the clients that have potential to commit fraud. Hence, we need a graph datastructure and algorithms to generate topological features for training a classification model.


## Approach

- Load Paysim dataset into Neo4j
- Preprocess the dataset 
  - add class labels to mule examples
  - generate additional relationships to add more context
  - build additional graphy features on client nodes 
- Generate Client embeddings using
  - Node2Vec (only relationships)
- Generate Train/Test Splits
- Train a Logistic Regression model in Neo4j GDS library
    - Use k-fold cross validation to compute model metrics
    - Test model performance of all models on the test graph
    - Pick the best model and predict if there are any mules among unlabelled clients using the trained LR model
- Get embeddings from Neo4j and train a neural net using Keras and compute model metrics

In [ ]:
#!/opt/anaconda3/bin/pip install neo4j
#!/opt/anaconda3/bin/pip install tensorflow
#!/opt/anaconda3/bin/pip install scikit-learn==0.24.2
#!/opt/anaconda3/bin/pip install pandas
import os
import numpy as np
import pandas as pd
from neo4j import GraphDatabase, exceptions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

## Neo4j interface
 
 Wrap neo4j python driver in Neo4j DB interface class

In [ ]:
from neo4j import GraphDatabase, Query, unit_of_work
class Neo4j():
    """
    Neo4J DB Interface class
    """
    def __init__(self, *args, **kwargs):
        url = kwargs.get('url', 'bolt://localhost:7687/')
        username = kwargs.get('username', 'neo4j')
        password = kwargs.get('password', 'letmein')
        database = kwargs.get('database', 'neo4j')
        self.client = GraphDatabase.driver(url, auth=(username, password), database=database)
        
    @unit_of_work(timeout=1200)
    def __run(self, tx, query, **kwargs):
        if kwargs.get('data_frame'):
            return pd.DataFrame([dict(record) for record in tx.run(query)])
        result = [row for row in tx.run(query)]
        return result
    
    def execute(self, query, **kwargs):
        with self.client.session() as session:
            tx = session.begin_transaction()
            result = self.__run(tx, query, **kwargs)
            tx.close()
            return result
        
    def read(self, query, **kwargs):
        with self.client.session() as session:
            return session.read_transaction(self.__run, query, **kwargs)
    
    def write(self, query):
        with self.client.session() as session:
            return session.write_transaction(self.__run, query)

In [ ]:
# Connect to Neo4j
url = 'neo4j+s://gds.neo4j.academy:443'
username = 'neo4j'
password = 'neo4j'
database = 'paysimxxx'
n = Neo4j(url=url, username=username, password=password, database=database)

## Preprocessing

### Add target property to identify Mules

In [ ]:
q = """MATCH (c:Client) WHERE NOT c:Mule SET c.is_mule = 0;"""
_ = n.write(q)

In [ ]:
q = """MATCH (c:Mule) SET c.is_mule = 1;"""
_ = n.write(q)

### `TRANSACTS_WITH` relationship 
- connect a pair of clients that send or receive money from others
- Instead of using actual amount of transaction, assign transaction categories based on transaction amounts
- Compute percentiles based on transaction amounts and assign transaction category

In [ ]:
# Bin transaction amounts into transaction categories
q = """
    MATCH (t:Transfer) 
    WITH apoc.agg.statistics(t.amount, [0.1, 0.25, 0.5, 0.75]) as m
    MATCH (t:Transfer) 
    WITH t, m, CASE 
        WHEN t.amount <= m.`0.1` THEN toFloat(1.0)
        WHEN m.`0.1` <= t.amount <= m.`0.25` THEN toFloat(2.0)
        WHEN m.`0.25` <= t.amount <= m.`0.5` THEN toFloat(3.0)
        WHEN m.`0.5` <= t.amount <= m.`0.75` THEN toFloat(4.0)
        WHEN t.amount > m.`0.75` THEN toFloat(5.0)
        ELSE toFloat(0.0)
    END AS cat
    SET t.amountCategory = cat;
    """
res = n.write(q)
print(res)

In [ ]:
# Create TRANSACTS_WITH relationship with transaction category as weight o the relationship
q = """
    CALL apoc.periodic.iterate(
    "MATCH (c1:Client)-[:PERFORMED]->(t:Transfer)-[:TO]->(c2:Client)
        RETURN c1, c2, t.amountCategory as txnCat",
    "MERGE (c1) - [:TRANSACTS_WITH {txnCat: txnCat}] -> (c2)",
    {batchSize: 1000});
    """
res = n.write(q)
print(res)

## Feature Engineering

### 1. Fraud risk score
- Compute fraud risk score for every client based on connections to previously identified fraudulent txns

In [ ]:
q = """
    MATCH (c:Client)-[]->(t:Transaction) WHERE t.fraud=true
    WITH count(t) as fraudtxns
    WITH apoc.coll.max(collect(fraudtxns)) as maxNum
    MATCH (c:Client)-[]->(t:Transaction) WHERE t.fraud=true
    WITH c, maxNum, count(t) as fraudCount
    SET  c.fraud_risk_score = toFloat(fraudCount) / maxNum;
"""
res = n.write(q)
print(res)

In [ ]:
# Set fraud_risk_score = 0 for all other clients
q = """
    MATCH (c:Client) WHERE NOT exists(c.fraud_risk_score)
    SET c.fraud_risk_score = 0.0
"""
res = n.write(q)
print(res)

### 2. Shared Identifiers Risk Score

Our hypothesis is that Clients who share identifiers (Phone Number, Email Address and SSN) are most likely to commit fraud. We exepct the clients not sharing any identifiers. It is possible a phone number is shared between two clients but a shared SSN is definitely a red flag.

**Examples**: 
- A client shares an email address with three other clients and a phone number with a different client and SSN with four clients.
- A client only shares a phone number with another client
- A clinet shares a SSN with three other clients
- ...

Compute **shared identifiers risk score** for all clients
- Find patterns of shared identifiers and persisting a new relationhsip between clients
- Find paiwise similarity between all clients that share identifiers 
- Use pairwise similarity score to compute weighted centrality score (**si_risk_score**)


In [ ]:
# relationship to connect clients that share one or more than one of the identifiers like SSN, Email or Phone Number
q = """
    MATCH (c1:Client)-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN] ->(n)<- [:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]-(c2:Client)
    WHERE c1.id<>c2.id
    WITH c1, c2, count(*) as cnt
    MERGE (c1) - [:SHARED_IDENTIFIERS {count: cnt}] -> (c2);
"""
res = n.write(q)
print(res)

In [ ]:
# Project a graph to run similarity 
q = """
    CALL gds.graph.create('client_graph', 
    'Client',
        {
            SHARED_IDENTIFIERS:{
                type: 'SHARED_IDENTIFIERS',
                orientation:'NATURAL',
                properties: 'count'
            }
        },
        {
            readConcurrency: 4
        }     
    ) YIELD graphName, nodeCount, relationshipCount, createMillis;
"""
res = n.execute(q, **{'data_frame':True})
res

In [ ]:
# Run Weighted Node Similarity and mutate in-memory graph

q = """
    CALL gds.nodeSimilarity.mutate('client_graph',
        {
            similarityCutoff: 0.05,
            concurrency: 4,
            mutateRelationshipType:'SIMILAR_TO',
            mutateProperty:'score',
            relationshipWeightProperty:'count'
        }                   
    )
"""
res = n.execute(q)
print(res)

In [ ]:
# Run weighted degree centrality to compute si_risk_score
q = """
    CALL gds.degree.mutate('client_graph',
        {
            nodeLabels: ['Client'],
            relationshipTypes:['SIMILAR_TO'],
            relationshipWeightProperty: 'score',
            mutateProperty: 'si_risk_score'
        }                   
    )
"""
res = n.execute(q)
print(res)

In [ ]:
# Write shared identifiers risk scores to the database
q = """
    CALL gds.graph.writeNodeProperties(
      'client_graph',
      ['si_risk_score'],
      ['Client'],
      { writeConcurrency: 4 }
    );
"""
res = n.execute(q, **{'data_frame':True})
res

In [ ]:
# Drop the in-memory graph
q = """ CALL gds.graph.drop('client_graph')"""
res = n.execute(q)
print(res)

In [ ]:
#Run this to fix convert the data type of risk scores from long to floats
q = """MATCH (c:Client) SET c.fraud_risk_score  = toFloat(c.fraud_risk_score);"""
_ = n.write(q)
q = """MATCH (c:Client) SET c.si_risk_score  = toFloat(c.si_risk_score);"""
_ = n.write(q)

### 3. Update `TRANSACTS_WITH` relationship

- Update the weight on the relationship with a composite score of transaction category + fraud_risk_score + si_risk_score
- This gives higher weight to relationships with known and suspected fraudsters.


In [ ]:
q = """
    CALL apoc.periodic.iterate(
        "MATCH (c1:Client) - [t:TRANSACTS_WITH] -> (c2:Client) RETURN c1, c2, t",
        "SET t.weight = t.txnCat + c1.fraud_risk_score + c1.si_risk_score + c2.fraud_risk_score + c2.si_risk_score",
        {batchSize:100}
    );
"""
res = n.execute(q)
print(res)

### Supervised ML: Train/Test Split

Label nodes to split Clients into Train and Test data

- Randomly splitting the dataset (80/20)
- Add new class labels to differentiate training and test data

In [ ]:
# Randomly pick 80% of clients and label them as training data
q = """
    CALL apoc.periodic.iterate(
    "MATCH (c:Client) WITH collect(ID(c)) as clients 
        RETURN apoc.coll.randomItems(clients, toInteger(0.8 * size(clients))) as trainClients", 
    "UNWIND trainClients as trainClient 
        MATCH (c:Client) WHERE ID(c) = trainClient 
        SET c.is_train_data = 1", {batchSize: 10000});
"""
res = n.write(q)
print(res)

In [ ]:
# Label the remianing clients as test data
q = """
MATCH (c:Client) WHERE NOT exists(c.is_train_data)
SET c.is_train_data = 0;
"""
res = n.write(q)
print(res)

In [ ]:
# Check the number of mules in train data

q = """
MATCH (c:Client) WHERE c.is_mule = 1 and c.is_train_data = 1
return count(c) as Mules;
"""
res = n.read(q, **{'data_frame': True})
res

In [ ]:
# Check the number of mules in test data

q = """
MATCH (c:Client) WHERE c.is_mule = 1 and c.is_train_data = 0
return count(c) as Mules;
"""
res = n.read(q, **{'data_frame': True})
res

## Client Embeddings

### Node2Vec
    - Node2Vec works on weighted graphs but doesn't take into account the node properties
    - Node2Vec is transductive; Embeddings have to be recomputed when the underlying data changes

In [ ]:
# Project in-memory graph with newly added relationships and node properties
q = """
    CALL gds.graph.create('mule_graph', 
        {
            Client:{
                label:'Client',
                properties:{
                    is_mule:{property:'is_mule',defaultValue:0},
                    fraud_risk_score:{property:'fraud_risk_score',defaultValue:0.0},
                    si_risk_score:{property: 'si_risk_score', defaultValue: 0.0},
                    is_train_data:{property:'is_train_data',defaultValue:0}
                }
            }
        },
        {
            TRANSACTS_WITH:{
                type: 'TRANSACTS_WITH',
                orientation:'UNDIRECTED',
                properties: 'weight',
                aggregation:'SUM'
            }
        },
        {
          readConcurrency: 4
        }
    ) YIELD graphName, nodeCount, relationshipCount, createMillis;
"""
res = n.execute(q)
print(res)

In [ ]:
# Generate embedding using Node2Vec 

q = """
    CALL gds.beta.node2vec.mutate('mule_graph', 
        {
         embeddingDimension: 256, 
         walkLength:16,
         walksPerNode: 64,
         returnFactor: 0.5,
         inOutFactor: 1.5,
         iterations: 20,
         mutateProperty:'n2v_embedding',
         concurrency:4,
         relationshipWeightProperty: 'weight'
        }
    );
"""
res = n.execute(q)
print(res)

In [ ]:
# Write Node embeddings back to the database
q = """
    CALL gds.graph.writeNodeProperties(
      'mule_graph',
      ['n2v_embedding'],
      ['Client'],
       {
         writeConcurrency: 4
       }
    );
"""
res = n.write(q)
print(res)

## Supervised ML in Neo4j

- Create a subgraph with train data
- Train a logistic regression model
- Save the model into model catalog

### 1. Train Graph

In [ ]:
# Create subgraph by filtering on `is_train_data` label
q = """
    CALL gds.beta.graph.create.subgraph('mule_train_graph', 'mule_graph', 'n:Client AND n.is_train_data = 1', '*')
    YIELD graphName, fromGraphName, nodeCount, relationshipCount;
"""
res = n.execute(q)
print(res)

In [ ]:
# Logistic regression model
q = """
    CALL gds.alpha.ml.nodeClassification.train('mule_train_graph', {
       nodeLabels: ['Client'],
       modelName: 'mule_model_n2v',
       featureProperties: ['n2v_embedding'], 
       targetProperty: 'is_mule', 
       metrics: ['F1_WEIGHTED', 'PRECISION(class=*)', 'RECALL(class=*)', 'F1(class=*)'], 
       holdoutFraction: 0.2, 
       validationFolds: 3, 
       randomSeed: 2,
       params: [
        {penalty: 0.0625},
        {penalty: 0.5},
        {penalty: 1.0}
       ]
    }) YIELD modelInfo
    RETURN
    {penalty: modelInfo.bestParameters.penalty} AS winningModel,
    modelInfo.metrics.F1_WEIGHTED.outerTrain AS F1_WEIGHTED_TRAIN,
    modelInfo.metrics.F1_WEIGHTED.test AS F1_WEIGHTED_TEST,
    modelInfo.metrics.F1_class_0.test AS F1_TEST_CLASS_0,
    modelInfo.metrics.F1_class_1.test AS F1_TEST_CLASS_1,
    modelInfo.metrics.PRECISION_class_0.test AS PRECISION_TEST_CLASS_0,
    modelInfo.metrics.PRECISION_class_1.test AS PRECISION_TEST_CLASS_1,
    modelInfo.metrics.RECALL_class_0.test AS RECALL_TEST_CLASS_0,
    modelInfo.metrics.RECALL_class_1.test AS RECALL_TEST_CLASS_1;
"""
res = n.execute(q, **{'data_frame':True})
res

### 2. Test Graph

- Predict mule labels and probability on test graph

In [ ]:
q = """
    CALL gds.beta.graph.create.subgraph('mule_test_graph', 'mule_graph', 'n:Client AND n.is_train_data = 0', '*')
    YIELD graphName, fromGraphName, nodeCount, relationshipCount;
"""
res = n.execute(q)
print(res)

In [ ]:
q = """
    CALL gds.alpha.ml.nodeClassification.predict.mutate('mule_test_graph', {
      nodeLabels: ['Client'],
      modelName: 'mule_model_n2v',
      mutateProperty: 'predicted_mule_n2v',
      predictedProbabilityProperty: 'predicted_mule_probability_n2v'
    });
"""
res = n.execute(q)
print(res)

### 3. Write predicted labels back to the database

In [ ]:
q = """
    CALL gds.graph.writeNodeProperties(
      'mule_test_graph',
      ['predicted_mule_n2v', 'predicted_mule_probability_n2v'],
      ['Client']
    );
"""
res = n.write(q)
print(res)

### 4. Evaluate the performance of the trained model

In [ ]:
q = """
    WITH [
      { model: 'Node2Vec', label: 'predicted_mule_n2v' }
      ] AS models
    UNWIND models AS model
    WITH model.model AS m, model.label AS label
    MATCH (c:Client) WHERE c[label] = 0 AND c.is_mule = 0
    WITH m, label, count(c) AS TP
    OPTIONAL MATCH (c:Client) WHERE c[label] = 0 AND c.is_mule = 1
    WITH m, label, TP, count(c) AS FN
    OPTIONAL MATCH (c:Client) WHERE c[label] = 1 AND c.is_mule = 0
    WITH m, label, TP, FN, count(c) AS FP
    OPTIONAL MATCH (c:Client) WHERE c[label] = 1 AND c.is_mule = 1
    WITH m, TP, FN, FP, count(c) AS TN
    RETURN m, TP, FP, FN, TN
"""
res = n.read(q)
print(res)

In [ ]:
cm = np.array([res[0][k] for k, v in res[0].items() if k != 'm']).reshape(2,2)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.array([0,1]))
disp.plot()

## Scikit-Learn/Tensorflow

- Get embeddings from Neo4j
- Train a neural network on train data
- Test the performance on test data

### 1. Embeddings from Neo4j

In [ ]:
q = """
MATCH (c:Client)
RETURN c.id AS ClientId, 
c.n2v_embedding AS Embedding,
c.is_mule as Mule,
c.is_train_data as Train_Test_Split;
"""
df = n.execute(q, **{'data_frame':True})
df

### 2. Train/Test Split

In [ ]:
# Train and Test data
train_df = df[df.Train_Test_Split == 1]
test_df = df[df.Train_Test_Split == 0]

X_train = np.vstack([row for row in train_df.Embedding.values])
Y_train = train_df.Mule.values

X_test = np.vstack([row for row in test_df.Embedding.values])
Y_test = test_df.Mule.values.tolist()

### 3. Tensorflow model

In [ ]:
def create_model(input_dim=256):
    # structure model
    model = Sequential()
    model.add(Dense(input_dim, input_dim=input_dim, activation="relu"))
    model.add(Dense(128, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    # compile model
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy", "Precision", "Recall"])
    return model

### 4. Train and Cross validate using scikit-learn Cross validation

In [ ]:
estimator = KerasClassifier(build_fn=create_model, epochs=100, batch_size=250, verbose=1)
kfold = StratifiedKFold(n_splits=3, shuffle=True)
results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
results
#print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

### 5. Model Training

In [ ]:
estimator.fit(X_train, Y_train)


### 6. Predict

In [ ]:
Y_pred = estimator.predict(X_test)
Y_pred = np.hstack([i for i in Y_pred])

### 7. Evaluate

In [ ]:
%matplotlib inline

In [ ]:
cm = confusion_matrix(Y_test, list(Y_pred), labels=estimator.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=estimator.classes_)
disp.plot()

## Cleanup

In [ ]:
# Drop in-memory graphs
q = """
    CALL gds.graph.list()
    YIELD graphName AS namedGraph, database
    WITH namedGraph where database='{database}'
    CALL gds.graph.drop(namedGraph)
    YIELD graphName
    RETURN graphName;
""".format(database=database)
res = n.execute(q)
print(res)


# Delete models
q = """
    CALL gds.beta.model.list()
    YIELD modelInfo
    WITH modelInfo.modelName as m
    CALL gds.beta.model.drop(m)
    YIELD modelInfo
    RETURN modelInfo.modelName;
"""
res = n.execute(q)
print(res)
                         
# Delete relationships
q = """
UNWIND ['SHARED_IDENTIFIERS', 'TRANSACTS_WITH'] AS rel
MATCH ()-[r]->() WHERE type(r) = rel
DELETE r;
"""
res = n.write(q)
print(res)

# Delete properties on clients
q = """
MATCH (c:Client) 
REMOVE c.is_mule, 
c.is_train_data, 
c.fraud_risk_score, 
c.si_risk_score, 
c.n2v_embedding,
c.predicted_mule_n2v,
c.predicted_mule_probability_n2v,
c.predicted_mule,
c.predicted_mule_probability;
"""
res = n.write(q)
print(res)

# Delete properties on Transfer nodes
q = """
MATCH (t:Transfer) REMOVE t.amountCategory;
"""
res = n.write(q)
print(res)